<a href="https://colab.research.google.com/github/ZZOMING-K/2024_Pytorch_Study/blob/main/2_%EC%84%A0%ED%98%95%EA%B3%84%EC%B8%B5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # 1. 행렬 곱 , Matrix Multiplication

 * 행렬 A와 B를 곱한다고 하자.
 * 행렬 요소 = 곱셈의 앞 행렬 A의 행 요소들을 행렬 B의 열의 요소들에 각각 곱한 후 더한 값
 * 따라서, **A의 열의 개수 = B의 행의 개수**
 * 행렬의 곱셈 과정 → Inner Product(내적), Dot Product(닷 프로덕트) 라고 부른다.  

In [19]:
import torch
import torch.nn as nn

In [ ]:
x = torch.FloatTensor([[1,2],
                      [3,4],
                      [5,6]])

y = torch.FloatTensor([[1,2],
                      [1,2]])

print(x.size() , y.size())

torch.Size([3, 2]) torch.Size([2, 2])


In [ ]:
z = torch.matmul(x,y)
print(z.size())

torch.Size([3, 2])


In [ ]:
print(z)

tensor([[ 3.,  6.],
        [ 7., 14.],
        [11., 22.]])


## 배치행렬곱, Batch Matrix Multiplication (BMM)
* 같은 갯수의 행렬 쌍들에 대해서 병렬로 행렬 곱 실행

In [ ]:
x = torch.FloatTensor(3,3,2)
y = torch.FloatTensor(3,2,3)

z = torch.bmm(x,y)
print(z.size())

torch.Size([3, 3, 3])


# 2.선형 계층, Linear Layer
* `심층 신경망의 가장 기본 구성 요소`
* Fully-Connected(FC) Layer 라고 불리기도 함
* 내부 가중치 파라미터(weight parameter) W와 b에 의해서 정의된다.

### Linear Layer 작동방식

* 각 입력 노드들에 `Weight(가중치)`를 곱하고 모두 합친 뒤, `bias(편향)`을 더한다.  

### Equations
* 행렬 곱으로 구현 가능
* n차원에서 m차원으로의 선형 변환 함수

## Raw Linear Layer

$$\begin{gathered}
y=x\cdot{W}+b, \\
\text{where }x\in\mathbb{R}^{N\times{n}}\text{, }y\in\mathbb{R}^{N\times{m}}. \\
\\
\text{Thus, }W\in\mathbb{R}^{n\times{m}}\text{ and }b\in\mathbb{R}^m.
\end{gathered}$$

In [ ]:
W= torch.FloatTensor([[1,2],
                     [3,4],
                     [5,6]])
b = torch.FloatTensor([2,2])
print(w.size() , b.size())

torch.Size([3, 2]) torch.Size([2])


In [ ]:
def linear(x, W, b) :
    y = torch.matmul(x,W) + b
    return y

In [ ]:
x = torch.FloatTensor(4,3)
print(x)

tensor([[-1.2296e+32,  1.2542e-42,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])


In [ ]:
y = linear(x,W,b)
print(y.size())

torch.Size([4, 2])


## nn.Module

In [ ]:
import torch.nn as nn

In [ ]:
class MyLinear(nn.Module) :
    def __init__(self, input_dim =3 , output_dim = 2) :
        self.input_dim = input_dim
        self.ouput_dim = output_dim

        super().__init__()

        self.W = torch.FloatTensor(input_dim , output_dim)
        self.b = torch.FloatTensor(output_dim)

    def forward(self, x) :
        # |x| = (batch_size , input_dim)
        y = torch.matmul(x, self.W) + self.b
        #|y| = (batch_size , input_dim) * (input_dim ,output_dim)
        #    = (batch_size , output_dim)
        return y

In [ ]:
linear = MyLinear(3,2)
y = linear(x)

In [ ]:
for p in linear.parameters() :
    print(p)

* nn.Module의 상속받는 객체는 __call__함수와 forward가 매핑되어 있어서 forward를 직접 부를 필요가 없다.

* nn.Module을 상속받은 선형 계층은 내부에 학습할 수 있는 파라미터는 없는 것으로 인식하기 때문에 위의 코드를 입력하면 아무것도 출력하지 않는다.

* W와 b를 파이토치에서 학습이 가능하도록 인식할 수 있는 파라미터를 만들어야함 -> `torch.nn.Parameter`


## Correct way : nn.Parameter

In [ ]:
class MyLinear(nn.Module) :
    def __init__(self, input_dim =3 , output_dim = 2) :
        self.input_dim = input_dim
        self.ouput_dim = output_dim

        super().__init__()

        self.W = nn.Parameter(torch.FloatTensor(input_dim , output_dim))
        self.b = nn.Parameter(torch.FloatTensor(output_dim))

    def forward(self, x) :
        # |x| = (batch_size , input_dim)
        y = torch.matmul(x, self.W) + self.b
        #|y| = (batch_size , input_dim) * (input_dim ,output_dim)
        #    = (batch_size , output_dim)
        return y

In [ ]:
linear = MyLinear(3,2)
y = linear(x)

In [ ]:
for p in linear.parameters() :
    print(p)

Parameter containing:
tensor([[-1.2174e+32,  1.2542e-42],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00]], requires_grad=True)
Parameter containing:
tensor([0., 0.], requires_grad=True)


## nn.Linear

In [ ]:
linear = nn.Linear(3,2)
y = linear(x)

In [ ]:
for p in linear.parameters() :
    print(p)

Parameter containing:
tensor([[-0.4247,  0.3154, -0.1926],
        [ 0.0358,  0.1518, -0.4883]], requires_grad=True)
Parameter containing:
tensor([ 0.0720, -0.1554], requires_grad=True)


### nn.Module can contain other nn.Module's child classes

In [ ]:
class MyLinear(nn.Module) :
    def __init__(self, input_dim = 3, output_dim = 2) :
        self.input_dim = input_dim
        self.ouput_dim = output_dim

        super().__init__()

        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self,x) :
        y = self.linear(x)
        return y

# 3. GPU 사용하기
* 딥러닝의 연산량은 엄청나기 때문에 딥러닝 연산은 보통 엔비디아 그래픽 처리 장치(GPU)에서 이루어진다.
* 파이토치에서도 CUDA 를 통한 GPU 연산을 지원한다.
* **서로 다른 장치에 올라가있는 텐서 또는 nn.Module의 하위 클래스 객체끼리는 연산이 불가능하다.**
* CPU 와 GPU에 위치한 텐서끼리 연산이 불가능할 뿐만 아니라 0번 GPU 그리고 1번 GPU 사이의 연산도 불가능하다.

### Convert to CUDA tensor : cuda()

In [3]:
x = torch.cuda.FloatTensor(2,2)
x

tensor([[0., 0.],
        [0., 0.]], device='cuda:0')

In [4]:
x = torch.FloatTensor(2,2)
x

tensor([[ 1.4013e-45,  0.0000e+00],
        [-3.7870e-21,  3.0674e-41]])

In [5]:
x.cuda()

tensor([[ 1.4013e-45,  0.0000e+00],
        [-3.7870e-21,  3.0674e-41]], device='cuda:0')

In [6]:
#device 지정
d= torch.device('cuda:0')

In [7]:
#move가 아닌 copy 이기 때문에 x 자체는 CPU
x.cuda(device =d)

tensor([[ 1.4013e-45,  0.0000e+00],
        [-3.7870e-21,  3.0674e-41]], device='cuda:0')

In [8]:
x.device

device(type='cpu')

### Convert to CUDA tensor : to()

In [9]:
x.to(device = d)

tensor([[ 1.4013e-45,  0.0000e+00],
        [-3.7870e-21,  3.0674e-41]], device='cuda:0')

## Convert to CPU tensor from CUDA tensor

In [14]:
x = torch.cuda.FloatTensor(2,2)
x

tensor([[ 1.4013e-45, -3.7870e-21],
        [ 3.0674e-41,  3.0674e-41]], device='cuda:0')

In [15]:
x = x.cpu()
x

tensor([[ 1.4013e-45, -3.7870e-21],
        [ 3.0674e-41,  3.0674e-41]])

In [16]:
d= torch.device('cpu')
x = x.to(d)
x

tensor([[ 1.4013e-45, -3.7870e-21],
        [ 3.0674e-41,  3.0674e-41]])

## Move Model from CPU to GPU

In [17]:
def print_params(model) :
    for p in model.parameters() :
        print(p)

In [20]:
linear = nn.Linear(2,2)
print_params(linear)

Parameter containing:
tensor([[ 0.6549,  0.2864],
        [-0.5833, -0.6752]], requires_grad=True)
Parameter containing:
tensor([-0.0830,  0.3626], requires_grad=True)


In [21]:
linear = linear.cuda()
print_params(linear)

Parameter containing:
tensor([[ 0.6549,  0.2864],
        [-0.5833, -0.6752]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.0830,  0.3626], device='cuda:0', requires_grad=True)


In [ ]:
linear = linear.cpu()
print_params(linear)

Parameter containing:
tensor([[-0.6543,  0.0342],
        [ 0.3232,  0.4173]], requires_grad=True)
Parameter containing:
tensor([-0.4891, -0.0220], requires_grad=True)


In [22]:
d = torch.device('cuda:0')
linear = linear.to(d)

print_params(linear)

Parameter containing:
tensor([[ 0.6549,  0.2864],
        [-0.5833, -0.6752]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.0830,  0.3626], device='cuda:0', requires_grad=True)


In [23]:
linear.device # 모델은 device 위치를 알 수 없음

AttributeError: 'Linear' object has no attribute 'device'

## Tricks

In [24]:
x = torch.cuda.FloatTensor(2,2)

In [25]:
#x와 같은 타입이면서 같은 디바이스에 있는 특정 사이즈의 텐서를 만들어줘
x.new(2,2)

tensor([[1.0842e-19, 1.4161e+00],
        [5.8335e-01, 6.7517e-01]], device='cuda:0')

In [26]:
torch.zeros_like(x)

tensor([[0., 0.],
        [0., 0.]], device='cuda:0')

In [27]:
torch.ones_like(x)

tensor([[1., 1.],
        [1., 1.]], device='cuda:0')

In [28]:
list(linear.parameters())[0].new(2,2)

tensor([[1., 1.],
        [1., 1.]], device='cuda:0')